In [78]:
import csv
import numpy as np
import pandas as pd

In [81]:
f_path = '/home/flink/workspace/yimin/datasets/nexmark_full_job_Nov14/test/test2.csv'
# ametrics = None
# with open(f_path) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     line_count = 0
#     for row in csv_reader:
#         if line_count == 0:
#             print(row)
#             line_count+=1

In [82]:
df = pd.read_csv(f_path)

In [220]:
class sequential_cleaner:
    def __init__(self, wanted_dict):
        self.wanted_run = wanted_dict.get('run_confs', None)
        self.wanted_job = wanted_dict.get('job_id', None)
        self.wanted_job_meta = wanted_dict.get('job_meta_info', None)
        self.wanted_job_level = wanted_dict.get('job_level_metrics', None) 
        self.wanted_vertex = wanted_dict.get('vertex_id', None)
        self.wanted_vertex_level = wanted_dict.get('vertex_level_metrics', None)
        self.wanted_sb_level = wanted_dict.get('sub_level_metrics', None)

        self.head = self.make_head()
        self.dataset = []
    
    def make_head(self,):
        res = []
        if self.wanted_run:
            res += self.wanted_run
        if self.wanted_job:
            res += ['job_id']
        if self.wanted_job_meta:
            res += self.wanted_job_meta
        if self.wanted_job_level:
            res += self.wanted_job_level
        if self.wanted_vertex:
            if self.wanted_vertex == 'idx':
                res += ['vertex_idx']
            else:
                res += ['vertex_id']
        if self.wanted_vertex_level:
            res += self.wanted_vertex_level
        if self.wanted_sb_level:
            res += ['sub_idx']
            res += self.wanted_sb_level
        return res
    
    def get_head(self,):
        return self.head

#     def help_select_wanted(self, attr_dict, want_list, sub_key = 'value'):
#         res = []
#         feasible_counter = 0
#         for k in attr_dict.keys():
#             if k in want_list:
#                 feasible_counter += 1
#                 if sub_key:
#                     res.append(attr_dict[k][sub_key])
#                 else:
#                     res.append(attr_dict[k])
#         if feasible_counter < len(want_list):
#             # means not all wanted is here, so skip
#             return []
#         return res
    
    def help_select_wanted(self, attr_dict, want_list, sub_key = 'value'):
        res = []
        feasible_counter = 0
        for k in want_list:
            if k in attr_dict.keys():
                feasible_counter += 1
                if sub_key:
                    res.append(attr_dict[k][sub_key])
                else:
                    res.append(attr_dict[k])
            else:
                break
        if feasible_counter < len(want_list):
            # means not all wanted is here, so skip
            return []
        return res
    
    def make_run_attributes(self, run_confs):
        return self.help_select_wanted(run_confs, self.wanted_run, '')
    
    def make_job_meta_attributes(self, job_metrics):
        return self.help_select_wanted(job_metrics['job_meta_info'], self.wanted_job_meta, '')
    
    def make_job_level_attributes(self, job_metrics):
        # TODO: needs further config info
        res = []
        job_metrics['job_level_metrics']
        for k,v in job_metrics.items():
            if k in self.wanted_job_level:
                res.append(max([e['value'] for e in v.values()]))
        return res
    
    def make_ver_level_attributes(self, job_metrics):
        dicts1 = job_metrics['job_meta_info']['vertices']
        dicts2 = job_metrics['vertice_metrics']
        v_num = len(dicts1)
        res1 = [self.help_select_wanted(dicts1[i], self.wanted_vertex_level, '') for i in range(v_num)]
        res2 = [self.help_select_wanted(dicts2[i]['vertex_level_metrics'], self.wanted_vertex_level, 'avg') for i in range(v_num)]
        res = []
        for i in range(v_num):
            res.append(res1[i] + res2[i])
        return res
    
    def make_sb_level_attributes(self, vertice_metrics):
        res = []
        for vm in vertice_metrics:
            v_res = []
            for sb_idx in range(len(vm['sub_level_metrics'])):
                v_res.append([sb_idx] + self.help_select_wanted(vm['sub_level_metrics'][sb_idx], self.wanted_sb_level))
            res.append(v_res)
        return res # 3-d array
    
    def assemble_rows(self, sub_rows_1=[], sub_rows_2=[], sub_rows_3=[]):
        # TODO: risky function, not robust yet
        res = [] # collected samples
        
        sub_rows_1_combined = []
        for r in sub_rows_1:
            sub_rows_1_combined += r
        
        sub_rows_2_combined = []
        for i in range(len(sub_rows_2[0] if sub_rows_2 else [])): # v num
            tmp = []
            for r in sub_rows_2:
                tmp += r[i]
            sub_rows_2_combined.append(tmp)
        
        # sub_rows_3_combined do not need to re-assemble
        
        # assumes there must be sub-task level metrics: (or, sub_rows_3 is not empty)
        for i in range(len(sub_rows_3)): # i->vertex
            for j in range(len(sub_rows_3[i])): # j->subtask
                a_sample = []
                a_sample += sub_rows_1_combined
                if sub_rows_2_combined:
                    sbr = sub_rows_2_combined[i]
                    if not sbr: continue
                    a_sample += sbr
                sbr = sub_rows_3[i][j]
                if not sbr: continue
                a_sample += sbr
                res.append(a_sample)
        return res # return a set of tabular samples
    
    def clean_row(self, row):
        run_confs = eval(row['run_confs']) # dict
        job_metrics = eval(row['job_metrics']) # dict
        
        cleaned_rows = []
        
        sub_rows_1 = [] # 1d
        sub_rows_2 = [] # 2d
        sub_rows_3 = [] # 3d
        
        if self.wanted_run:
            run_attrs = self.make_run_attributes(run_confs) # 1-list
            sub_rows_1.append(run_attrs)
        if self.wanted_job:
            job_attrs = [job_metrics['job_id']]
            sub_rows_1.append(job_attrs)
        if self.wanted_job_meta:
            job_meta_attrs = self.make_job_meta_attributes(job_metrics)
            sub_rows_1.append(job_meta_attrs)
        if self.wanted_job_level:
            job_level_attrs = self.make_job_level_attributes(job_metrics)
            sub_rows_1.append(job_level_attrs)
        
        if self.wanted_vertex:
            if self.wanted_vertex == 'idx':
                ver_attrs = [[i] for i in range(len(job_metrics['vertice_metrics']))]
            else:
                ver_attrs = [[e['vertex_id']] for e in job_metrics['vertice_metrics']]
            sub_rows_2.append(ver_attrs)
        if self.wanted_vertex_level:
            ver_level_attrs = self.make_ver_level_attributes(job_metrics)
            sub_rows_2.append(ver_level_attrs)

        if self.wanted_sb_level:
            sb_level_attrs = self.make_sb_level_attributes(job_metrics['vertice_metrics'])
            sub_rows_3.append(sb_level_attrs)

        # debug
#         print(sub_rows_3[0], '\n')
        res = self.assemble_rows(sub_rows_1, sub_rows_2, sub_rows_3[0])
#         print(res, '\n\n')
        self.dataset += res
        return row

In [232]:
wanted_dict = {}
wanted_dict['run_confs'] = ['qid']
# wanted_dict['job_id'] = 1
wanted_dict['vertex_level_metrics'] = ['parallelism',]
wanted_dict['vertex_id'] = 'idx'
wanted_dict['sub_level_metrics'] = ['busyTimeMsPerSecond', 'Shuffle.Netty.Input.Buffers.inputQueueLength',
                                    'Shuffle.Netty.Input.Buffers.inputQueueSize', 'Shuffle.Netty.Input.Buffers.inPoolUsage',
                                    'numBytesInPerSecond', 'numRecordsInPerSecond',]
scleaner = sequential_cleaner(wanted_dict)
df[150:160].apply(scleaner.clean_row, axis=1)
scleaner.dataset[:10]

[[9, 0, 2, 0, 'NaN', '0', '0', '0.0', '0.0', '0.0'],
 [9, 0, 2, 1, 'NaN', '0', '0', '0.0', '0.0', '0.0'],
 [9, 1, 2, 0, '957.0', '21', '0', '0.0', '1334750.1333333333', '5562.4'],
 [9,
  1,
  2,
  1,
  '679.0',
  '4',
  '0',
  '0.0',
  '1330381.0666666667',
  '5527.916666666667'],
 [9,
  2,
  2,
  0,
  '0.0',
  '2',
  '0',
  '0.0',
  '2603979.3666666667',
  '3908.516666666667'],
 [9,
  2,
  2,
  1,
  '0.0',
  '2',
  '0',
  '0.0',
  '2478367.183333333',
  '3688.7833333333333'],
 [9, 0, 2, 0, 'NaN', '0', '0', '0.0', '0.0', '0.0'],
 [9, 0, 2, 1, 'NaN', '0', '0', '0.0', '0.0', '0.0'],
 [9, 1, 2, 0, '919.0', '21', '0', '0.0', '3356821.283333333', '13984.7'],
 [9, 1, 2, 1, '975.0', '4', '0', '0.0', '3338385.9166666665', '13879.65']]

In [233]:
scleaner.get_head()

['qid',
 'vertex_id',
 'parallelism',
 'sub_idx',
 'busyTimeMsPerSecond',
 'Shuffle.Netty.Input.Buffers.inputQueueLength',
 'Shuffle.Netty.Input.Buffers.inputQueueSize',
 'Shuffle.Netty.Input.Buffers.inPoolUsage',
 'numBytesInPerSecond',
 'numRecordsInPerSecond']

In [132]:
debugg = eval(df['job_metrics'][0])

In [199]:
debugg['vertice_metrics'][1]['sub_level_metrics'][1]
# debugg['job_meta_info']['vertices'][0]

{'checkpointStartDelayNanos': {'value': '0'},
 'Shuffle.Netty.Output.Buffers.outPoolUsage': {'value': '0.0076923077'},
 'numBytesInLocal': {'value': '191413'},
 'checkpointAlignmentTime': {'value': '0'},
 'numBytesInRemotePerSecond': {'value': '0.0'},
 'Shuffle.Netty.Input.numBytesInRemotePerSecond': {'value': '0.0'},
 'maxSoftBackPressureTimeMs': {'value': '0'},
 'numBytesOut': {'value': '1540096'},
 'buffers.inputQueueSize': {'value': '0'},
 'buffers.outputQueueSize': {'value': '1468'},
 'numBytesIn': {'value': '2625806'},
 'numBuffersOut': {'value': '47'},
 'Shuffle.Netty.Input.numBuffersInLocal': {'value': '258'},
 'numBuffersInRemotePerSecond': {'value': '0.0'},
 'numBytesOutPerSecond': {'value': '19660.8'},
 'buffers.outputQueueLength': {'value': '2'},
 'numBuffersOutPerSecond': {'value': '0.6'},
 'Shuffle.Netty.Input.Buffers.inputExclusiveBuffersUsage': {'value': '0.0'},
 'isBackPressured': {'value': 'false'},
 'softBackPressuredTimeMsPerSecond': {'value': '0'},
 'numBytesInLoca

In [70]:
for i in range(1):
    print(debugg['vertice_metrics'][i]['vertex_level_metrics']['numBytesInPerSecond'])

{'min': 0.0, 'max': 0.0, 'avg': 0.0, 'sum': 0.0}


In [108]:
['1']+[]

['1']

In [176]:
if 'Nan':
    print(1)

1
